<a href="https://colab.research.google.com/github/Nayoung-young/prj-ecommerce-eda/blob/main/%EC%85%80%EB%9F%AC_%EC%9C%A0%EC%A0%80_%EC%84%B1%EA%B3%BC_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 유저 성과 분석 (4번)

In [3]:
# 라이브러리 임포트

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd


# _1. 데이터 전처리

## 1.1. 데이터셋 불러오기

In [29]:
%%time
df_re_org = pd.read_csv('/content/drive/MyDrive/dataset_2023/패스트캠퍼스/bootcamp-python-project/olist_order_reviews_dataset.csv')

df_customer_org = pd.read_csv('/content/drive/MyDrive/dataset_2023/패스트캠퍼스/bootcamp-python-project/olist_customers_dataset.csv')
df_ord_org = pd.read_csv('/content/drive/MyDrive/dataset_2023/패스트캠퍼스/bootcamp-python-project/olist_orders_dataset.csv')
df_it_org = pd.read_csv('/content/drive/MyDrive/dataset_2023/패스트캠퍼스/bootcamp-python-project/olist_order_items_dataset.csv')

df_seller_org = pd.read_csv('/content/drive/MyDrive/dataset_2023/패스트캠퍼스/bootcamp-python-project/olist_sellers_dataset.csv')

In [7]:
df_re_org.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                99224 non-null  object
 1   order_id                 99224 non-null  object
 2   review_score             99224 non-null  int64 
 3   review_comment_title     11568 non-null  object
 4   review_comment_message   40977 non-null  object
 5   review_creation_date     99224 non-null  object
 6   review_answer_timestamp  99224 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


In [8]:
df_customer_org.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [35]:
df_ord_org.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [44]:
# df_ord_org[['order_approved_at', 'order_delivered_carrier_date']].isna().sum()

In [43]:
# df_ord_org[ df_ord_org[['order_approved_at', 'order_delivered_carrier_date']].isna() ]

In [32]:
df_it_org.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


## 1.2. 전처리

In [70]:
# df_ord 전처리

df_ord = df_ord_org.copy()

print('='* 50 +'\n')
print('df_ord\n')
print(f'전처리 전: {df_ord.shape}')

## 1) 필요한 컬럼만 남기기

df_ord = df_ord_org[['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp']]

## 2) order_status == 'delivered'

df_ord = df_ord[ df_ord['order_status'] == 'delivered' ]

## 3) order_purchase_year, month 추가

df_ord['order_purchase_timestamp'] = pd.to_datetime(df_ord['order_purchase_timestamp'])

df_ord['order_purchase_date'] = df_ord['order_purchase_timestamp'].dt.date
df_ord['order_purchase_year'] = df_ord['order_purchase_timestamp'].dt.year
df_ord['order_purchase_month'] = df_ord['order_purchase_timestamp'].dt.month


# df_ord.drop(columns = ['order_purchase_timestamp'], inplace=True)

## 4) order_purchase_year == 2016 인 경우 제외

df_ord = df_ord[ df_ord['order_purchase_year'] != 2016 ]

print(f'전처리 후: {df_ord.shape}\n')
print('='* 50 +'\n')

print(df_ord.info())


df_ord

전처리 전: (99441, 8)
전처리 후: (96211, 7)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 96211 entries, 0 to 99440
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   order_id                  96211 non-null  object        
 1   customer_id               96211 non-null  object        
 2   order_status              96211 non-null  object        
 3   order_purchase_timestamp  96211 non-null  datetime64[ns]
 4   order_purchase_date       96211 non-null  object        
 5   order_purchase_year       96211 non-null  int64         
 6   order_purchase_month      96211 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 5.9+ MB
None


In [71]:
df_ord.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_purchase_date,order_purchase_year,order_purchase_month
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02,2017,10
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-24,2018,7
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08,2018,8
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18,2017,11
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13,2018,2


In [24]:
# df_re 전처리

df_re = df_re_org.copy()

print('='* 50 +'\n')
print('df_re\n')
print(f'전처리 전: {df_re.shape}')

## 1) 필요한 컬럼만 남기기

df_re = df_re_org[['review_id', 'order_id', 'review_score']]

print(f'전처리 후: {df_re.shape}\n')
print('='* 50 +'\n')

print(df_re.info())



df_re

전처리 전: (99224, 7)
전처리 후: (99224, 3)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   review_id     99224 non-null  object
 1   order_id      99224 non-null  object
 2   review_score  99224 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 2.3+ MB
None


In [27]:
# df_customer 전처리

df_customer = df_customer_org.copy()

print('='* 50 +'\n')
print('df_customer\n')
print(f'전처리 전: {df_customer.shape}')

## 1) 필요한 컬럼만 남기기

df_customer = df_customer_org[['customer_id', 'customer_unique_id', 'customer_state']]

print(f'전처리 후: {df_customer.shape}\n')
print('='* 50 +'\n')

print(df_customer.info())



df_customer

전처리 전: (99441, 5)
전처리 후: (99441, 3)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         99441 non-null  object
 1   customer_unique_id  99441 non-null  object
 2   customer_state      99441 non-null  object
dtypes: object(3)
memory usage: 2.3+ MB
None


In [34]:
# df_it 전처리

df_it = df_it_org.copy()

print('='* 50 +'\n')
print('df_it\n')
print(f'전처리 전: {df_it.shape}')

## 1) 필요한 컬럼만 남기기

df_it = df_it_org[['order_id', 'order_item_id', 'product_id', 'seller_id', 'price']]

print(f'전처리 후: {df_it.shape}\n')
print('='* 50 +'\n')

print(df_it.info())



df_it

전처리 전: (112650, 7)
전처리 후: (112650, 5)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   order_id       112650 non-null  object 
 1   order_item_id  112650 non-null  int64  
 2   product_id     112650 non-null  object 
 3   seller_id      112650 non-null  object 
 4   price          112650 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 4.3+ MB
None


#_2. 시각화

## 2.1. 어떤 유저가 2018년에 가장 많이 구매했는가?

> 목적: VIP 유저 리스트 추출하기

* 리스트 출력:
  * customer_unique_id | 2018년 총 상품 구매 금액 | 2018년 첫 상품 구매 | 2018년 마지막 구매 | 2018년 구매 빈도 수

* 시각화:  
  * 구매 금액별 유저 count? (히스토그램)
  * 구매 금액 x 축 / 구매 빈도 수 y 축 : 유저 클러스터링

### 2.1.1 데이터 준비 (merge)
- orders(order_id, order_purchase_year, month, day)  - customers(customer_unique_id) - order_items (price)

In [52]:
# df_ord.info()

In [65]:
# df_ord와 df_customer customer_id 기준 inner join

print('='* 50 +'\n')
print(f'merge 전 - df_ord: {df_ord.shape}')
print(f'merge 전 - df_customer: {df_customer.shape}\n')

print('='* 50 +'\n')

df_ord_customer = pd.merge(df_ord, df_customer, on ='customer_id', how ='inner') # 주문 내역이 있는 customer만 남음

print(f'merge 후 - df_ord_customer: {df_ord_customer.shape}\n')

df_ord_customer.info()



merge 전 - df_ord: (96211, 7)
merge 전 - df_customer: (99441, 3)


merge 후 - df_ord_customer: (96211, 9)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96211 entries, 0 to 96210
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   order_id                  96211 non-null  object        
 1   customer_id               96211 non-null  object        
 2   order_status              96211 non-null  object        
 3   order_purchase_timestamp  96211 non-null  datetime64[ns]
 4   order_purchase_year       96211 non-null  int64         
 5   order_purchase_month      96211 non-null  int64         
 6   order_purchase_day        96211 non-null  int64         
 7   customer_unique_id        96211 non-null  object        
 8   customer_state            96211 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 7.3+ MB


In [63]:
# df_ord_customer와 df_it를 order_id 기준 inner join

print('='* 50 +'\n')
print(f'merge 전 - df_ord_customer: {df_ord_customer.shape}')
print(f'merge 전 - df_it: {df_it.shape}\n')

print('='* 50 +'\n')

df_ord_customer_it = pd.merge(df_ord_customer, df_it, on ='order_id', how ='inner')

print(f'merge 후 - df_ord_customer_it: {df_ord_customer_it.shape}\n')

df_ord_customer_it.info()



merge 전 - df_ord_customer: (96211, 9)
merge 전 - df_it: (112650, 5)


merge 후 - df_ord_customer_it: (109880, 13)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109880 entries, 0 to 109879
Data columns (total 13 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   order_id                  109880 non-null  object        
 1   customer_id               109880 non-null  object        
 2   order_status              109880 non-null  object        
 3   order_purchase_timestamp  109880 non-null  datetime64[ns]
 4   order_purchase_year       109880 non-null  int64         
 5   order_purchase_month      109880 non-null  int64         
 6   order_purchase_day        109880 non-null  int64         
 7   customer_unique_id        109880 non-null  object        
 8   customer_state            109880 non-null  object        
 9   order_item_id             109880 non-null  int64         
 10  product_id    

In [64]:
df_ord_customer_it[['order_id', 'order_item_id']].nunique()

order_id         96211
order_item_id       21
dtype: int64

In [60]:
# 필요한 컬럼만 추출하기

df_ord_customer_it = df_ord_customer_it[['order_id', 'customer_unique_id', 'order_item_id', 'price','order_purchase_year', 'order_purchase_month', 'order_purchase_day']]
df_ord_customer_it.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109880 entries, 0 to 109879
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              109880 non-null  object 
 1   customer_unique_id    109880 non-null  object 
 2   order_item_id         109880 non-null  int64  
 3   price                 109880 non-null  float64
 4   order_purchase_year   109880 non-null  int64  
 5   order_purchase_month  109880 non-null  int64  
 6   order_purchase_day    109880 non-null  int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 6.7+ MB


In [61]:
df_ord_customer_it.head() # 이 상태에서 customer_unique_id 기준으로 집계

,order_id,customer_unique_id,order_item_id,price,order_purchase_year,order_purchase_month,order_purchase_day
0,e481f51cbdc54678b7cc49136f2d6af7,7c396fd4830fd04220f754e42b4e5bff,1,29.99,2017,10,2
1,53cdb2fc8bc7dce0b6741e2150273451,af07308b275d755c9edb36a90c618231,1,118.70,2018,7,24
2,47770eb9100c2d0c44946d9cf07ec65d,3a653a41f6f9fc3d2a113cf8398680e8,1,159.90,2018,8,8
3,949d5b44dbf5de918fe9c16f97b45f8a,7c142cf63193a1473d2e66489a9ae977,1,45.00,2017,11,18
4,ad21c59c0840e6cb83a9ceb5573f8159,72632f0f9dd73dfee390c9b22eb56dd6,1,19.90,2018,2,13


### 2.2.2 집계 (df 가공)
* 리스트 출력:
  * customer_unique_id | 2018년 총 상품 구매 금액 | 2018년 첫 상품 구매 | 2018년 마지막 구매 | 2018년 구매 빈도 수

In [66]:
df_ord_customer_it.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109880 entries, 0 to 109879
Data columns (total 13 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   order_id                  109880 non-null  object        
 1   customer_id               109880 non-null  object        
 2   order_status              109880 non-null  object        
 3   order_purchase_timestamp  109880 non-null  datetime64[ns]
 4   order_purchase_year       109880 non-null  int64         
 5   order_purchase_month      109880 non-null  int64         
 6   order_purchase_day        109880 non-null  int64         
 7   customer_unique_id        109880 non-null  object        
 8   customer_state            109880 non-null  object        
 9   order_item_id             109880 non-null  int64         
 10  product_id                109880 non-null  object        
 11  seller_id                 109880 non-null  object        
 12  pr

In [61]:
# 2018년 VIP 고객

df_vip_2018 = df_ord_customer_it[ df_ord_customer_it['order_purchase_year']  == 2018 ]



In [61]:
# 2017년 VIP 고객

df_vip_2017 = df_ord_customer_it[ df_ord_customer_it['order_purchase_year']  == 2017 ]



### 2.2.3 시각화

## 2.2. 어떤 유저가 2018년에 가장 많이 구매했는가?

# _3. 결론